In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

In [42]:
police_path = '/Users/John/Desktop/DS340W/DS340_ProjectCode/Police_Stations.csv'
restroom_path = '/Users/John/Desktop/DS340W/DS340_ProjectCode/Public_Restrooms.csv'
df = pd.read_csv(police_path)
df2 = pd.read_csv(restroom_path)

df.head(5)

,Facility Name,"Lat, long"
0,New York City Police Department - 32nd Precinct,"40.81573360901537, -73.94535716562918"
1,New York City Police Department - 109th Precinct,"40.76231630544898, -73.82728132549775"
2,New York City Police Department - 79th Precinct,"40.68885327486311, -73.94486367546305"
3,New York City Police Department - 90th Precinct,"40.70635526624754, -73.950563082209"
4,New York City Police Department - 84th Precinct,"40.69557453693269, -73.98291153558192"


In [ ]:
# Split 'Lat, long' into two columns 'latitude' and 'longitude'
df[['Latitude', 'Longitude']] = df['Lat, long'].str.split(',', expand=True)

# Convert the new columns to numeric types
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])

# Display the resulting DataFrame
df


In [ ]:
nyc_zipcode = gpd.read_file('./Modified Zip Code Tabulation Areas (MODZCTA)_20241012.geojson')

df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
df_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

df_gdf_zip = gpd.sjoin(df_gdf, nyc_zipcode, how='left')

df_gdf_zip['zipcode'] = df_gdf_zip['modzcta']

df_gdf_zip.drop(['label', 'zcta', 'index_right', 'modzcta'], axis = 1)

In [48]:
nyc_zipcode = gpd.read_file('./Modified Zip Code Tabulation Areas (MODZCTA)_20241012.geojson')

df2['geometry'] = df2.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
df_gdf2 = gpd.GeoDataFrame(df2, geometry='geometry', crs="EPSG:4326")

df_gdf_zip2 = gpd.sjoin(df_gdf2, nyc_zipcode, how='left')

df_gdf_zip2['zipcode'] = df_gdf_zip2['modzcta']

df_gdf_zip2 = df_gdf_zip2.drop(['label', 'zcta', 'index_right', 'modzcta'], axis = 1)

In [60]:
a = df_gdf_zip2['zipcode'].isna().sum()
b = (df_gdf_zip2['zipcode'] == '99999').sum()
a + b

49

In [11]:
# Assuming your DataFrame is named 'df'
# Group by 'zipcode' and count the number of 'Facility Name' in each zipcode
facility_count_by_zipcode = df_gdf_zip.groupby('zipcode')['Facility Name'].count().reset_index()

# Rename the count column for clarity
facility_count_by_zipcode.columns = ['zipcode', 'facility_count']

facility_count_by_zipcode.head()

facility_count_by_zipcode.to_csv('/Users/John/Desktop/DS340W/data/number_of_ps_within_zipcode.csv')



In [61]:
# Assuming your DataFrame is named 'df'
# Group by 'zipcode' and count the number of 'Facility Name' in each zipcode
facility_count_by_zipcode2 = df_gdf_zip2.groupby('zipcode')['Facility Name'].count().reset_index()

# Rename the count column for clarity
facility_count_by_zipcode2.columns = ['zipcode', 'facility_count']

facility_count_by_zipcode2.head()

facility_count_by_zipcode2.to_csv('/Users/John/Desktop/DS340W/data/number_of_restroom_within_zipcode.csv')



In [16]:
# Increase facility_count by 1 for zipcode 10024
facility_count_by_zipcode.loc[facility_count_by_zipcode['zipcode'] == '10024', 'facility_count'] += 1

# Delete the row where zipcode is '99999'
facility_count_by_zipcode = facility_count_by_zipcode[facility_count_by_zipcode['zipcode'] != '99999']

# Display the updated DataFrame
facility_count_by_zipcode


,zipcode,facility_count
0,10001,1
1,10002,1
2,10003,2
3,10004,1
4,10009,1
...,...,...
78,11428,1
79,11432,1
80,11434,1
81,11693,1


In [64]:
facility_count_by_zipcode2 = facility_count_by_zipcode2[facility_count_by_zipcode2['zipcode'] != '99999']

In [29]:
nypd_complaint = pd.read_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Cleaned_Zipcode.csv', dtype={'zipcode': 'object'})

/var/folders/r6/qcs_flrd7fj2bg8ml8rp61lc0000gn/T/ipykernel_7116/4237884846.py:1: DtypeWarning: Columns (1,38) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_complaint = pd.read_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Cleaned_Zipcode.csv', dtype={'zipcode': 'object'})


In [30]:
nypd_complaint

,Unnamed: 0,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,...,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,CMPLNT_FR_DAY,geometry,index_right,modzcta,label,zcta,pop_est,zipcode
0,0,280510523,14,MANHATTAN,01/11/2024,04:00:00,NaN,NaN,COMPLETED,0,...,2024,1,11,POINT (-73.9945368920152 40.7535327012632),16.0,10018.0,10018,10018,9687.0,10018
1,1,286736978,122,STATEN ISLAND,01/01/2015,13:00:00,01/01/2015,15:00:00,COMPLETED,0,...,2015,1,1,POINT (-74.1059144051212 40.5742856881925),53.0,10306.0,10306,10306,53142.0,10306
2,2,289019347,1,MANHATTAN,06/20/2024,16:12:00,06/20/2024,16:13:00,COMPLETED,1,...,2024,6,20,POINT (-74.00643483276845 40.70955616028041),34.0,10038.0,10038,10038,22800.0,10038
3,3,283688417,121,STATEN ISLAND,03/11/2024,10:00:00,03/13/2024,13:20:00,COMPLETED,0,...,2024,3,11,POINT (-74.165142 40.634542),49.0,10303.0,10303,10303,25947.0,10303
4,4,280934552,122,STATEN ISLAND,11/14/2023,12:00:00,11/17/2023,12:00:00,COMPLETED,0,...,2023,11,14,POINT (-74.155293 40.537874),58.0,10312.0,10312,10312,61741.0,10312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282085,282064,285642938,105,QUEENS,04/13/2024,14:25:00,04/13/2024,14:40:00,COMPLETED,0,...,2024,4,13,POINT (-73.757935 40.658385),157.0,11413.0,11413,11413,42443.0,11413
282086,282065,283438384,113,QUEENS,03/08/2024,12:30:00,03/08/2024,12:40:00,COMPLETED,0,...,2024,3,8,POINT (-73.753709 40.679481),157.0,11413.0,11413,11413,42443.0,11413
282087,282066,283855204,105,QUEENS,03/15/2024,18:00:00,03/16/2024,14:00:00,COMPLETED,0,...,2024,3,15,POINT (-73.718494 40.731658),163.0,11426.0,11426,11426,20801.0,11426
282088,282067,288669905,111,QUEENS,06/17/2024,23:40:00,06/18/2024,00:15:00,COMPLETED,0,...,2024,6,17,POINT (-73.76296 40.744204),138.0,11364.0,11364,11364,36806.0,11364


In [21]:
facility_count_by_zipcode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83 entries, 0 to 82
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   zipcode         83 non-null     object
 1   facility_count  83 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
df_merged = pd.merge(nypd_complaint, facility_count_by_zipcode, on='zipcode', how='left')
df_merged['facility_count'] = df_merged['facility_count'].fillna(0)
df_merged = df_merged.rename(columns={'facility_count': 'number_of_ps'})
df_merged = df_merged.drop(['label', 'zcta', 'index_right', 'modzcta', 'New Georeferenced Column', 'Lat_Lon'], axis = 1)
df_merged.head()

In [68]:
df_merged2 = pd.merge(df_merged, facility_count_by_zipcode2, on='zipcode', how='left')
df_merged2['facility_count'] = df_merged2['facility_count'].fillna(0)
df_merged2 = df_merged2.rename(columns={'number_of_ps': 'num_policestation'})
df_merged2 = df_merged2.rename(columns={'facility_count': 'num_restroom'})
# df_merged2 = df_merged2.drop(['label', 'zcta', 'index_right', 'modzcta', 'New Georeferenced Column', 'Lat_Lon'], axis = 1)
df_merged2.head()

,Unnamed: 0,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,...,Latitude,Longitude,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,CMPLNT_FR_DAY,geometry,pop_est,zipcode,num_policestation,num_restroom
0,0,280510523,14,MANHATTAN,01/11/2024,04:00:00,NaN,NaN,COMPLETED,0,...,40.753533,-73.994537,2024,1,11,POINT (-73.9945368920152 40.7535327012632),9687.0,10018,1.0,0.0
1,1,286736978,122,STATEN ISLAND,01/01/2015,13:00:00,01/01/2015,15:00:00,COMPLETED,0,...,40.574286,-74.105914,2015,1,1,POINT (-74.1059144051212 40.5742856881925),53142.0,10306,1.0,9.0
2,2,289019347,1,MANHATTAN,06/20/2024,16:12:00,06/20/2024,16:13:00,COMPLETED,1,...,40.709556,-74.006435,2024,6,20,POINT (-74.00643483276845 40.70955616028041),22800.0,10038,1.0,5.0
3,3,283688417,121,STATEN ISLAND,03/11/2024,10:00:00,03/13/2024,13:20:00,COMPLETED,0,...,40.634542,-74.165142,2024,3,11,POINT (-74.165142 40.634542),25947.0,10303,0.0,2.0
4,4,280934552,122,STATEN ISLAND,11/14/2023,12:00:00,11/17/2023,12:00:00,COMPLETED,0,...,40.537874,-74.155293,2023,11,14,POINT (-74.155293 40.537874),61741.0,10312,0.0,7.0


In [41]:
# df_merged.to_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Cleaned_Zipcode_Police.csv')

In [69]:
df_merged2.to_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Final.csv')